In [ ]:
# Parameters for the ZSL algorithm
# as defined in: https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl.ZSLGenerator
MAX_AREA = 400
MAX_AREA_TOL = 0.09
MAX_LENGTH_TOL = 0.03
MAX_ANGLE_TOL = 0.01

# Strains within this tolerance are considered equal
STRAIN_TOL = 10e-6

# Select materials
SUBSTRATE_INDEX = 0
LAYER_INDEX = 1

# Miller indices and thickness for substrate and layer.
SUBSTRATE_MILLER = (1, 1, 1)
SUBSTRATE_THICKNESS = 1
LAYER_MILLER = (0, 0, 1)
LAYER_THICKNESS = 1

# Distance between layers, in Angstroms
DISTANCE = 3.0


# Plot settings
# Strain axis limits in percent
X_MIN = 0.01
X_MAX = 100

# Number of atoms axis limits
Y_MIN = 1
Y_MAX = 1000

In [ ]:
from install_packages import install_packages

await install_packages()

In [ ]:
from definitions import get_materials, set_materials, to_pymatgen, from_pymatgen, plot_strain_vs_atoms

In [ ]:
# Load materials from Materials Designer list (in ESSE Material Config format, JSON) to `data` variable
get_materials()

In [ ]:
print(f"Loaded {len(data)} materials from Materials Designer:")
pymatgen_materials = [to_pymatgen(item) for item in data]
for material in pymatgen_materials:
    print(material)

In [ ]:
from pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from operator import itemgetter

# Select materials from the list of input materials
SUBSTRATE_MATERIAL = pymatgen_materials[SUBSTRATE_INDEX]
LAYER_MATERIAL = pymatgen_materials[LAYER_INDEX]


def main():
    # Create Interface Builder class
    zsl = ZSLGenerator(
        max_area_ratio_tol=MAX_AREA_TOL,
        max_area=MAX_AREA,
        max_length_tol=MAX_LENGTH_TOL,
        max_angle_tol=MAX_ANGLE_TOL,
    )
    cib = CoherentInterfaceBuilder(
        substrate_structure=SUBSTRATE_MATERIAL,
        film_structure=LAYER_MATERIAL,
        substrate_miller=SUBSTRATE_MILLER,
        film_miller=LAYER_MILLER,
        zslgen=zsl,
    )

    # Run the Interface Building process
    cib._find_terminations()
    matches = cib.zsl_matches
    terminations = cib.terminations

    # Create interfaces
    interfaces = []
    for termination in terminations:
        interfaces = list(
            cib.get_interfaces(
                termination,
                gap=DISTANCE,
                film_thickness=LAYER_THICKNESS,
                substrate_thickness=SUBSTRATE_THICKNESS,
                in_layers=True,
            )
        )

    print(f"Found {len(matches)} interfaces")
    print(f"Found {len(terminations)} terminations:", terminations)

    strain_modes = {
        "VON_MISES": "von_mises_strain",
        "STRAIN": "strain",
        "MEAN": "mean_abs_strain",
    }
    strain_mode = strain_modes["MEAN"]
    interfaces_list = list(interfaces)

    # Sort interfaces by ascending strain and then by ascending number of atoms
    sorted_interfaces = sorted(
        interfaces_list,
        key=lambda x: (itemgetter(strain_mode)(x), x["interface"].num_sites),
    )

    print("Interface with lowest strain (index 0):")
    print("    strain:", sorted_interfaces[0][strain_mode] * 100, "%")
    print("    number of atoms:", sorted_interfaces[0]["interface"].num_sites)

    # plot stran vs number of atoms via matplotlib
    plot_strain_vs_atoms(strain_mode, sorted_interfaces)
    return sorted_interfaces


interfaces = main()

In [ ]:
m3_esse = from_pymatgen(interfaces[0]["interface"])
print(m3_esse)

In [ ]:
# Submit materials (JSON config format) back to Materials Designer
set_materials([m3_esse])